# Prompt Engineering: Structure and Elements

This notebook explores the fundamental structures and elements of prompts when working with Large Language Models (LLMs), with a focus on recruiting scenarios.

In [3]:
import json
from subprocess import Popen, PIPE
from IPython.display import Markdown, display

def query_model(prompt, system_prompt=None, **kwargs):
    """Query Ollama with a specific presence_penalty setting"""
    data = {
        "model": "llama3",
        "prompt": prompt,
        "options": { **kwargs }
    }

    if system_prompt:
        data["system"] = system_prompt

    cmd = [
        "curl",
        "http://localhost:11434/api/generate",
        "-d",
        json.dumps(data)
    ]

    process = Popen(cmd, stdout=PIPE, stderr=PIPE)
    output, _ = process.communicate()

    responses = [json.loads(line) for line in output.decode().strip().split("\n")]
    return "".join(r.get("response", "") for r in responses)

def display_example(title, prompt, system_prompt=None):
    """Display an example prompt and its response"""
    display(Markdown(f"### {title}"))
    display(Markdown(f"**Prompt:**\n{prompt}"))
    if system_prompt:
        display(Markdown(f"**System Prompt:**\n{system_prompt}"))
    response = query_model(prompt, system_prompt)
    display(Markdown(f"**Response:**\n{response}"))

## 1. Core Elements of Prompts

A well-structured prompt typically consists of several key elements:

1. **Task Description**
   - Clear statement of what you want the model to do
   - Specific instructions about the desired output format
   - Example: "Analyze this resume and extract key skills"

2. **Context**
   - Background information needed for the task
   - Relevant constraints or requirements
   - Example: "You are an experienced technical recruiter reviewing candidates for a senior software engineer position"

3. **Input Data**
   - The specific content to be processed
   - Can be text, structured data, or examples
   - Example: A job description or resume text

4. **Output Format**
   - Desired structure of the response
   - Can be JSON, bullet points, paragraphs, etc.
   - Example: "Return the analysis in JSON format with skills, experience, and education sections"

5. **Examples (Few-shot Learning)**
   - Sample inputs and desired outputs
   - Helps model understand the expected format
   - Example: Showing a sample resume analysis before processing a new one

6. **Constraints and Guidelines**
   - Specific limitations or requirements
   - Style preferences or tone
   - Example: "Keep the response concise and professional"

7. **Chain of Thought**
   - Breaking down complex tasks into steps
   - Explaining reasoning process
   - Example: "First analyze technical skills, then assess experience level, finally evaluate culture fit"


## 2. Basic Prompt Structure Example

Let's start with a basic prompt structure for job description analysis:

In [4]:
# Basic prompt structure example
job_description = '''
Senior Software Engineer
Company: TechCorp
Location: Remote

We are seeking a Senior Software Engineer with strong experience in cloud technologies and distributed systems.

Requirements:
- 5+ years of experience in software development
- Expertise in Python and JavaScript
- Experience with AWS or Azure
- Strong understanding of microservices architecture
'''

basic_prompt = f'''Task: Analyze this job description and identify key requirements.
Input: {job_description}
Output Format: Please provide a structured list of:
1. Required years of experience
2. Technical skills required
3. Type of role (remote/onsite)
'''

display_example("Basic Job Description Analysis", basic_prompt)

### Basic Job Description Analysis

**Prompt:**
Task: Analyze this job description and identify key requirements.
Input: 
Senior Software Engineer
Company: TechCorp
Location: Remote

We are seeking a Senior Software Engineer with strong experience in cloud technologies and distributed systems.

Requirements:
- 5+ years of experience in software development
- Expertise in Python and JavaScript
- Experience with AWS or Azure
- Strong understanding of microservices architecture

Output Format: Please provide a structured list of:
1. Required years of experience
2. Technical skills required
3. Type of role (remote/onsite)


**Response:**
Here is the analysis of the job description:

**Required years of experience:** 5+ years of experience in software development

**Technical skills required:**

* Expertise in Python and JavaScript
* Experience with AWS or Azure
* Strong understanding of microservices architecture

**Type of role:** Remote

## 3. Context-Enhanced Prompts

Adding context helps the model understand the perspective and requirements better:

In [5]:
# Context-enhanced prompt example
system_prompt = '''You are an experienced technical recruiter with expertise in:
- Software engineering roles
- Technical skill evaluation
- Industry standards and market trends
Your goal is to provide detailed, actionable insights for hiring managers.'''

context_prompt = f'''Context: You are helping a fast-growing startup evaluate job requirements for a senior engineering role.
They want to ensure the requirements are competitive but not overly restrictive.

Task: Analyze this job description and provide recommendations for improvement.
Input: {job_description}

Please provide:
1. Analysis of current requirements
2. Suggestions for additional important requirements
3. Recommendations for making the job description more attractive
4. Potential red flags or overly restrictive requirements'''

display_example("Context-Enhanced Job Analysis", context_prompt, system_prompt)

### Context-Enhanced Job Analysis

**Prompt:**
Context: You are helping a fast-growing startup evaluate job requirements for a senior engineering role.
They want to ensure the requirements are competitive but not overly restrictive.

Task: Analyze this job description and provide recommendations for improvement.
Input: 
Senior Software Engineer
Company: TechCorp
Location: Remote

We are seeking a Senior Software Engineer with strong experience in cloud technologies and distributed systems.

Requirements:
- 5+ years of experience in software development
- Expertise in Python and JavaScript
- Experience with AWS or Azure
- Strong understanding of microservices architecture


Please provide:
1. Analysis of current requirements
2. Suggestions for additional important requirements
3. Recommendations for making the job description more attractive
4. Potential red flags or overly restrictive requirements

**System Prompt:**
You are an experienced technical recruiter with expertise in:
- Software engineering roles
- Technical skill evaluation
- Industry standards and market trends
Your goal is to provide detailed, actionable insights for hiring managers.

**Response:**
Analysis:

The current job description is straightforward and clearly defines the required skills and experience for a Senior Software Engineer role. However, there are some areas that could be improved to make it more competitive and inclusive.

Suggestions for additional important requirements:

1. Add "Strong understanding of containerization (e.g., Docker)" as this is an essential skill in modern software development.
2. Consider adding "Experience with CI/CD pipelines" as this ensures the candidate has hands-on experience with automating testing, building, and deployment processes.
3. Include "Knowledge of DevOps practices and tools" to ensure the candidate is familiar with collaboration, version control, and continuous improvement.

Recommendations for making the job description more attractive:

1. Emphasize the company culture: Highlight TechCorp's values, mission, and unique aspects that would appeal to top talent.
2. Add a section on growth opportunities: Mention training programs, mentorship, or career development initiatives to show your commitment to employee growth.
3. Showcase the project scope: Provide an overview of the team's current projects, technologies used, and future directions to give candidates a sense of what they'll be working on.

Potential red flags or overly restrictive requirements:

1. The requirement for "5+ years of experience" might be too narrow; consider adjusting it to "3-5 years of experience with a strong portfolio" to attract more diverse candidates.
2. The emphasis on AWS and Azure might be too specific; consider adding "Cloud-agnostic mindset" or "Experience with cloud computing (AWS, Azure, GCP, etc.)" to show flexibility in the team's stack choices.

Revised job description:

Senior Software Engineer
Company: TechCorp
Location: Remote

We are seeking a Senior Software Engineer with strong experience in cloud technologies and distributed systems. As part of our innovative team, you'll have the opportunity to work on cutting-edge projects that drive business growth.

Requirements:

* 3-5 years of experience in software development with a strong portfolio
* Expertise in Python and JavaScript
* Experience with AWS, Azure, or Google Cloud Platform
* Strong understanding of microservices architecture
* Knowledge of containerization (e.g., Docker)
* Familiarity with CI/CD pipelines and DevOps practices

Additional highlights:

* Emphasis on collaboration, innovation, and continuous learning
* Opportunities for mentorship, training, and career growth
* Exposure to a wide range of technologies and projects
* Flexible work arrangements and remote-friendly culture

By incorporating these recommendations, you'll create a job description that showcases your company's unique value proposition while being competitive in the market.

## 4. Few-Shot Learning with Examples

Providing examples helps the model understand the expected format and style:

In [13]:
# Few-shot learning example
resume_text = '''
JOHN DOE
Software Engineer

EXPERIENCE
Senior Developer, TechCorp (2019-Present)
- Led development of cloud-native applications using Python and AWS
- Managed team of 5 developers for microservices migration

Software Engineer, StartupX (2017-2019)
- Developed full-stack applications using React and Node.js
- Implemented CI/CD pipelines using Jenkins

EDUCATION
BS Computer Science, Tech University (2017)
'''

few_shot_prompt = '''Here's how to analyze a resume:

Example Resume:
---
Jane Smith
Full-Stack Developer

Experience:
Senior Engineer, WebCo (2018-2020)
- Built scalable web applications
- Led team of 3 developers

Education:
MS Computer Science, State University
---

Example Analysis:
{{
  "years_experience": 2,
  "role_level": "Senior",
  "key_skills": ["Web Development", "Team Leadership"],
  "education_level": "Masters"
}}

Now analyze this resume:
{resume}

Provide the analysis in the same JSON format.'''.format(resume=resume_text)

display_example("Few-Shot Resume Analysis", few_shot_prompt)

### Few-Shot Resume Analysis

**Prompt:**
Here's how to analyze a resume:

Example Resume:
---
Jane Smith
Full-Stack Developer

Experience:
Senior Engineer, WebCo (2018-2020)
- Built scalable web applications
- Led team of 3 developers

Education:
MS Computer Science, State University
---

Example Analysis:
{
  "years_experience": 2,
  "role_level": "Senior",
  "key_skills": ["Web Development", "Team Leadership"],
  "education_level": "Masters"
}

Now analyze this resume:

JOHN DOE
Software Engineer

EXPERIENCE
Senior Developer, TechCorp (2019-Present)
- Led development of cloud-native applications using Python and AWS
- Managed team of 5 developers for microservices migration

Software Engineer, StartupX (2017-2019)
- Developed full-stack applications using React and Node.js
- Implemented CI/CD pipelines using Jenkins

EDUCATION
BS Computer Science, Tech University (2017)


Provide the analysis in the same JSON format.

**Response:**
Here's the analysis of John Doe's resume:

{
  "years_experience": 4,
  "role_level": "Senior",
  "key_skills": ["Cloud-Native Development", "Team Leadership", "Python", "AWS", "Full-Stack Development", "React", "Node.js", "CI/CD Pipelines"],
  "education_level": "Bachelors"
}

Note that I've extracted the following information:

* Years of experience: John has been in the industry for 4 years, with 2 years as a senior developer.
* Role level: As mentioned earlier, he's a senior developer.
* Key skills: I've listed all the technologies and skills mentioned in his work experience, including cloud-native development, team leadership, and specific programming languages and tools.
* Education level: John holds a Bachelor's degree in Computer Science.

## 5. Chain of Thought Prompts

Breaking down complex tasks into steps helps achieve better results:

In [14]:
# Chain of thought example
company_info = '''
TechCorp
Industry: Enterprise Software
Size: 500+ employees
Culture: Fast-paced, innovative, remote-first
Benefits: Competitive salary, equity, unlimited PTO
Current Challenges: Scaling engineering team, improving product deployment speed
'''

cot_prompt = f'''Let's analyze this company's hiring needs step by step.

Input Company Information:
{company_info}

Please follow these steps:
1. First, analyze the company size and industry to understand the context
2. Then, evaluate the company culture and what it means for hiring
3. Next, consider the benefits package and its competitiveness
4. After that, examine the current challenges
5. Finally, provide specific recommendations for:
   a) Type of candidates to target
   b) Key skills to prioritize
   c) Cultural attributes to look for
   d) Suggested improvements to benefits package

For each step, explain your reasoning before moving to the next step.'''

display_example("Chain of Thought Company Analysis", cot_prompt)

### Chain of Thought Company Analysis

**Prompt:**
Let's analyze this company's hiring needs step by step.

Input Company Information:

TechCorp
Industry: Enterprise Software
Size: 500+ employees
Culture: Fast-paced, innovative, remote-first
Benefits: Competitive salary, equity, unlimited PTO
Current Challenges: Scaling engineering team, improving product deployment speed


Please follow these steps:
1. First, analyze the company size and industry to understand the context
2. Then, evaluate the company culture and what it means for hiring
3. Next, consider the benefits package and its competitiveness
4. After that, examine the current challenges
5. Finally, provide specific recommendations for:
   a) Type of candidates to target
   b) Key skills to prioritize
   c) Cultural attributes to look for
   d) Suggested improvements to benefits package

For each step, explain your reasoning before moving to the next step.

**Response:**
Let's analyze TechCorp's hiring needs step by step.

**Step 1: Analyze company size and industry**

TechCorp has over 500 employees, which indicates a mid-to-large-sized organization. This suggests that they may have established processes and systems in place, but also may require more complex organizational structures to manage their growth. Additionally, being an Enterprise Software company means that they likely deal with large-scale projects and require specialized skills.

Reasoning: Understanding the company size gives us insight into their current capacity for growth and the complexity of their operations. Knowing the industry provides context for the types of skills and expertise required in their engineering team.

**Next step: Evaluate company culture**

Please let me know when you're ready to proceed!

## 6. Constrained Output Format

Specifying exact output formats helps get consistent, structured responses:

In [15]:
# Constrained output format example
job_req = '''
Full-Stack Developer
Required Skills:
- React, Node.js, MongoDB
- 3+ years experience
- Agile development
Nice to Have:
- AWS experience
- TypeScript
- Team leadership
'''

structured_prompt = f'''Analyze this job requirement and return a JSON object with the following structure:
{{
  "role_name": "string",
  "required_skills": ["string"],
  "minimum_years": "number",
  "nice_to_have": ["string"],
  "level": "string",
  "suggested_interview_questions": ["string"]
}}

Input:
{job_req}

Ensure the response is valid JSON and includes exactly these fields.'''

display_example("Structured Job Requirements Analysis", structured_prompt)

### Structured Job Requirements Analysis

**Prompt:**
Analyze this job requirement and return a JSON object with the following structure:
{
  "role_name": "string",
  "required_skills": ["string"],
  "minimum_years": "number",
  "nice_to_have": ["string"],
  "level": "string",
  "suggested_interview_questions": ["string"]
}

Input:

Full-Stack Developer
Required Skills:
- React, Node.js, MongoDB
- 3+ years experience
- Agile development
Nice to Have:
- AWS experience
- TypeScript
- Team leadership


Ensure the response is valid JSON and includes exactly these fields.

**Response:**
Here's the JSON object:

```
{
  "role_name": "Full-Stack Developer",
  "required_skills": ["React", "Node.js", "MongoDB"],
  "minimum_years": 3,
  "nice_to_have": ["AWS experience", "TypeScript", "Team leadership"],
  "level": "",
  "suggested_interview_questions": []
}
```

Let me know if this meets your expectations!

## 7. Best Practices for Prompt Engineering

1. **Be Specific and Clear**
   - Use precise language
   - Break down complex tasks
   - Specify output format

2. **Provide Context**
   - Include relevant background information
   - Specify the role or perspective
   - Mention important constraints

3. **Use Examples Effectively**
   - Provide clear input/output examples
   - Use few-shot learning for complex tasks
   - Show edge cases when relevant

4. **Structure Your Prompts**
   - Start with clear task description
   - Include all necessary elements
   - Use consistent formatting

5. **Iterate and Refine**
   - Test with different inputs
   - Adjust based on results
   - Keep track of successful patterns

6. **Consider the Model's Limitations**
   - Stay within context window
   - Break up large tasks
   - Verify critical information